# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data.head(-1)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,6.06,87,75,5.66,CL,1627422196
1,Faanui,-16.4833,-151.7500,25.57,74,25,3.44,PF,1627422196
2,Ponta do Sol,32.6667,-17.1000,23.34,71,8,2.77,PT,1627422197
3,Hilo,19.7297,-155.0900,25.85,78,90,3.60,US,1627422197
4,Bethel,41.3712,-73.4140,31.64,57,20,0.45,US,1627422197
...,...,...,...,...,...,...,...,...,...
573,Sterling,39.0062,-77.4286,35.50,37,20,0.45,US,1627422097
574,Itaituba,-4.2761,-55.9836,34.09,52,20,0.00,BR,1627422337
575,Ahipara,-35.1667,173.1667,14.51,83,44,3.15,NZ,1627422338
576,Nanortalik,60.1432,-45.2371,8.14,76,55,2.02,GL,1627422338


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 21) & (weather_data["Max Temp"] <= 26)].dropna()
narrowed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,Touros,-5.1989,-35.4608,25.50,82,0,5.96,BR,1627422186
147,Tiznit Province,29.5833,-9.5000,21.52,72,0,0.99,MA,1627422232
158,East London,-33.0153,27.9116,22.84,26,0,4.49,ZA,1627421953
180,Kazygurt,41.7589,69.3864,25.96,33,0,4.17,KZ,1627422239
189,Huéscar,37.8110,-2.5412,24.41,61,0,1.54,ES,1627422241
193,Kaseda-shirakame,31.4167,130.3167,22.57,78,0,1.46,JP,1627422241
218,Tortolì,39.9256,9.6572,24.99,78,0,5.66,IT,1627422247
245,Falāvarjān,32.5553,51.5097,23.82,21,0,3.11,IR,1627422253
383,Athabasca,54.7169,-113.2854,24.23,17,0,7.07,CA,1627422286
477,Tūkrah,32.5341,20.5791,25.19,77,0,6.16,LY,1627422313


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
123,Touros,BR,-5.1989,-35.4608,
147,Tiznit Province,MA,29.5833,-9.5000,
158,East London,ZA,-33.0153,27.9116,
180,Kazygurt,KZ,41.7589,69.3864,
189,Huéscar,ES,37.8110,-2.5412,
193,Kaseda-shirakame,JP,31.4167,130.3167,
218,Tortolì,IT,39.9256,9.6572,
245,Falāvarjān,IR,32.5553,51.5097,
383,Athabasca,CA,54.7169,-113.2854,
477,Tūkrah,LY,32.5341,20.5791,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 123: Touros.
Closest hotel in Touros is Hotel Vila Gale Touros.
------------
Retrieving Results for Index 147: Tiznit Province.
Missing field/result... skipping.
------------
Retrieving Results for Index 158: East London.
Closest hotel in East London is Blue Lagoon Hotel.
------------
Retrieving Results for Index 180: Kazygurt.
Missing field/result... skipping.
------------
Retrieving Results for Index 189: Huéscar.
Closest hotel in Huéscar is Hotel El Maño.
------------
Retrieving Results for Index 193: Kaseda-shirakame.
Closest hotel in Kaseda-shirakame is Sun Stay Kaseda.
------------
Retrieving Results for Index 218: Tortolì.
Closest hotel in Tortolì is Hotel La Bitta.
------------
Retrieving Results for Index 245: Falāvarjān.
Closest hotel in Falāvarjān is رضا سیف الله.
------------
Retrieving Results for Index 383: Athabasca.
Closest hotel in Athabasca is Days Inn by Wyndham Athabasca.
------------
Retrieving Results for Index 477: Tūkrah.
Closest hot

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
123,Touros,BR,-5.1989,-35.4608,Hotel Vila Gale Touros
147,Tiznit Province,MA,29.5833,-9.5000,
158,East London,ZA,-33.0153,27.9116,Blue Lagoon Hotel
180,Kazygurt,KZ,41.7589,69.3864,
189,Huéscar,ES,37.8110,-2.5412,Hotel El Maño
193,Kaseda-shirakame,JP,31.4167,130.3167,Sun Stay Kaseda
218,Tortolì,IT,39.9256,9.6572,Hotel La Bitta
245,Falāvarjān,IR,32.5553,51.5097,رضا سیف الله
383,Athabasca,CA,54.7169,-113.2854,Days Inn by Wyndham Athabasca
477,Tūkrah,LY,32.5341,20.5791,مصيف مرحبا للعائلات


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))